# Verify Deployment and DB Specs, Counts etc..

1 = Personal or Desktop Engine 2 = Standard 3 = Enterprise 4 = Express 5 = SQL Database 6 = SQL Data Warehouse 8 = SQL Managed Instance

[https://docs.microsoft.com/en-us/azure/azure-sql/database/monitoring-with-dmvs](https://docs.microsoft.com/en-us/azure/azure-sql/database/monitoring-with-dmvs)

In [ ]:
--What Version am I running
SELECT @@VERSION 
--What Engine Edition
SELECT SERVERPROPERTY('EngineEdition');
--Bring back DB name, creation date, compatibility level, collation
SELECT * FROM sys.databases;


In [ ]:
/* verify server memory configuration options */
SELECT * FROM sys.configurations
WHERE [name] IN ('max server memory (mb)',  'min server memory (mb)');

In [ ]:
--Gets back db name, dtu limits, cpu limits, memory limits, size limits, log limits etc
SELECT * FROM sys.dm_user_db_resource_governance -- Only available in Azure SQL DB and MI

In [ ]:
/* verify installed physical memory, memory usage, memory model, service startup time */
SELECT	
	CEILING(physical_memory_kb / 1024.0 / 1024.0) AS physical_memory_GB,
	CEILING(committed_kb / 1024.0) AS committed_memory_MB,
	CEILING(committed_target_kb / 1024.0) AS committed_target_memory_MB,
	CEILING(visible_target_kb / 1024.0) AS visible_target_memory_MB,
	sql_memory_model_desc,
	sqlserver_start_time
FROM sys.dm_os_sys_info;

In [ ]:
/* verify databases, COMPATIBILITY LEVELS, recovery models, if snapshot isolation modes are enabled */
SELECT [name],database_id,[state_desc],[compatibility_level],recovery_model_desc,snapshot_isolation_state_desc,is_read_committed_snapshot_on
FROM sys.databases;

In [ ]:
--Run this to get the size of the DB(s) in SQL Server
SELECT
    name,
    size,
    size * 8/1024 'Size (MB)',
    max_size
FROM sys.master_files
WHERE DB_NAME(database_id) = 'WideWorldImporters';


SELECT
	database_id,
	[file_id],
	[type],
	[type_desc],
	[name],
	physical_name,
	size,
	max_size,
	is_percent_growth,
	growth	
FROM sys.master_files
WHERE database_id = DB_ID('AdventureWorks');


SELECT
    d.name AS 'Database',
    m.name AS 'File',
    m.size,
    m.size * 8/1024 'Size (MB)',
    SUM(m.size * 8/1024) OVER (PARTITION BY d.name) AS 'Database Total',
    m.max_size
FROM sys.master_files m
INNER JOIN sys.databases d ON
d.database_id = m.database_id;

In [ ]:
--Get table names of a particular database
SELECT TABLE_NAME
FROM INFORMATION_SCHEMA.TABLES
WHERE TABLE_TYPE='BASE TABLE'

In [ ]:
--Capture the Row Counts for all tables in a database
SELECT
      QUOTENAME(SCHEMA_NAME(sOBJ.schema_id)) + '.' + QUOTENAME(sOBJ.name) AS [TableName]
      , SUM(sdmvPTNS.row_count) AS [RowCount]
FROM
      sys.objects AS sOBJ
      INNER JOIN sys.dm_db_partition_stats AS sdmvPTNS
            ON sOBJ.object_id = sdmvPTNS.object_id
WHERE 
      sOBJ.type = 'U'
      AND sOBJ.is_ms_shipped = 0x0
      AND sdmvPTNS.index_id < 2
GROUP BY
      sOBJ.schema_id
      , sOBJ.name
ORDER BY [TableName]
GO

In [ ]:
--Query to list all Stored Procedures and Functions
SELECT name, type
FROM dbo.sysobjects
WHERE type IN (
    'P', -- stored procedures
    'FN', -- scalar functions 
    'IF', -- inline table-valued functions
    'TF' -- table-valued functions
)
ORDER BY type, name